<a href="https://colab.research.google.com/github/AzeemWaqarRao/Pytorch_Implementations/blob/main/CNNusingPytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torchvision.datasets import MNIST
from torch import nn
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader,random_split
from torch.optim import Adam


In [ ]:
# Preparing Data

In [ ]:
dataset = MNIST(root = './data', train = True, transform = ToTensor(), download=True)

100%|██████████| 9912422/9912422 [00:00<00:00, 134624163.76it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 60416804.90it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 36474489.06it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw


100%|██████████| 4542/4542 [00:00<00:00, 5099177.94it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [ ]:
train_set, test_set = random_split(dataset, [50000, 10000])

In [ ]:
train_data = DataLoader(train_set,32)
test_data = DataLoader(test_set,32)

In [ ]:
len(train_set), len(test_set)

(50000, 10000)

In [ ]:
X,y = [batch for batch in train_data][0]

In [ ]:
y

tensor([1, 0, 5, 0, 9, 7, 0, 8, 1, 5, 3, 5, 9, 9, 3, 4, 7, 8, 4, 1, 5, 2, 2, 2,
        4, 6, 5, 1, 2, 2, 6, 1])

In [ ]:
# Creating Neural Network

In [ ]:
from torch.nn.modules.activation import ReLU
class CNN(nn.Module):
  def __init__(self):
    super().__init__()
    self.model = nn.Sequential(
        nn.Conv2d(1,32,(3,3)),
        nn.ReLU(),
        nn.Conv2d(32,64,(3,3)),
        nn.ReLU(),
        nn.Conv2d(64,64,(3,3)),
        nn.ReLU(),
        nn.Flatten(),
        nn.Linear(64*22*22,10)
    )

  def forward(self,x):
    return self.model(x)

In [ ]:
cnn = CNN().to('cuda')

In [ ]:
loss = nn.CrossEntropyLoss()
optimizer = Adam(cnn.parameters(),lr=1e-3)

In [ ]:
for epoch in range(10):
  for batch in train_data:
    X,y = batch
    X,y = X.to('cuda'), y.to('cuda')
    yhat = cnn(X)
    loss_val = loss(yhat,y)

    # apply backprop
    optimizer.zero_grad()
    loss_val.backward()
    optimizer.step()
  print(f"epoch {epoch} , Loss {loss_val.item()}")

epoch 0 , Loss 0.0628684014081955
epoch 1 , Loss 0.0035062895622104406
epoch 2 , Loss 0.03281376510858536
epoch 3 , Loss 7.847746019251645e-05
epoch 4 , Loss 0.003355341963469982
epoch 5 , Loss 0.000566033530049026
epoch 6 , Loss 0.00026751254335977137
epoch 7 , Loss 1.2731760762108024e-05
epoch 8 , Loss 6.934305565664545e-05
epoch 9 , Loss 0.0055708568543195724


In [ ]:
results = torch.tensor([]).to('cuda')
actual = torch.tensor([]).to('cuda')
for batch in test_data:
  X,y = batch
  X,y = X.to('cuda'), y.to('cuda')
  yhat = cnn(X)
  pred = (yhat.argmax(axis=1))
  results = torch.cat((results,pred))
  actual = torch.cat((actual,y))


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
cf = confusion_matrix(results.tolist(),actual.tolist())

In [ ]:
print(cf)

[[ 979    0    1    0    2    1   10    0    2    2]
 [   0 1100    5    1    1    2    1    4    0    1]
 [   1    3  995    5    1    0    0    6    1    1]
 [   0    0    2  948    0    5    0    0    2    0]
 [   0    0    1    0  985    0    2    2    0    6]
 [   0    0    0    2    1  887    0    0    0    0]
 [   3    0    1    0    2    7  991    0    0    1]
 [   1    2    6    4    1    0    0 1077    2    1]
 [   2    2    4    2    3    6    5    1  944    8]
 [   1    1    0    0    4    2    1    3    5  939]]


In [ ]:
cr = classification_report(results.tolist(),actual.tolist())
print(cr)

              precision    recall  f1-score   support

         0.0       0.99      0.98      0.99       997
         1.0       0.99      0.99      0.99      1115
         2.0       0.98      0.98      0.98      1013
         3.0       0.99      0.99      0.99       957
         4.0       0.98      0.99      0.99       996
         5.0       0.97      1.00      0.99       890
         6.0       0.98      0.99      0.98      1005
         7.0       0.99      0.98      0.98      1094
         8.0       0.99      0.97      0.98       977
         9.0       0.98      0.98      0.98       956

    accuracy                           0.98     10000
   macro avg       0.98      0.98      0.98     10000
weighted avg       0.98      0.98      0.98     10000



In [ ]:
torch.save(cnn,'cnn.pt')